# Microcalcification cluster creation
This notebook contains the functions used to create microcalcification, microcalcification clusters, placement of clusters in breast phantom, and preprocessing of Monte-Carlo GPU simulations.

To convert any calcification or cluster into a raw file, use: 

`cluster_type.tofile('cluster.raw')`

In [1]:
import numpy as np
import random as rand
import math
import pathlib
#!pip install raster_geometry
from raster_geometry import * #For creating spherical calcifications
#!pip install bresenham
from bresenham import bresenham #For creating linear clusters
from calc_cluster_generation import * #File in this repo

## Creating individual calcifications repalce with importing the python file
### Spherical calcifications
Example clusters are placed in folder `/calc_cluster_examples`.

In [6]:
calc_size = 9
num_rmv = 10
calc = create_calc(calc_size, num_rmv)
calc = calc.astype('uint8')
calc.tofile('./calc_cluster_examples/calc{0}x{0}_rmv{1}.raw'.format(calc_size, num_rmv))

### Rod-like calcifications

In [7]:
rod_size = 7
rod_num_rmv = 7**2
rod = create_calc_rod(rod_size, rod_num_rmv)
calc_type = rod.astype('uint8')
calc_type.tofile('./calc_cluster_examples/calc_rod{0}x{0}_rmv{1}.raw'.format(rod_size, rod_num_rmv))

## Creating microcalcification clusters
### Random clusters

In [8]:
cluster_size = 142
num_calcs = 20
#Default min_calc_size = 3, max_calc_size = 9
cluster = create_cluster(cluster_size, num_calcs)
cluster.tofile('./calc_cluster_examples/cluster{0}x{0}_nm{1}.raw'.format(cluster_size, num_calcs))

### Random non-uniform clusters

In [9]:
nonuniform_cluster_size = 142
nonuniform_num_calcs = 20
#Default min_calc_size = 3, max_calc_size = 9
#Default weights = [0.2, 0.2, 0.2, 0.2, 0.03, 0.03, 0.03], where the larger sizes, 7-9 voxels, have a probability of 10%. 
nonuniform_cluster = create_cluster_nonuniform(cluster_size, num_calcs)
nonuniform_cluster.tofile('./calc_cluster_examples/cluster_nonuniform{0}x{0}_nm{1}.raw'.format(cluster_size, num_calcs))

### Linear clusters

In [10]:
linear_cluster_size = 142
linear_num_calcs = 20
linear_cluster = create_cluster_linear(linear_cluster_size, linear_num_calcs, min_calc_size = 3, max_calc_size = 9, 
                                                               num_away_min = 15, num_away_max = 30)
linear_cluster.tofile('./calc_cluster_examples/cluster_linear{0}x{0}_nm{1}_{2}_m.raw'.format(linear_cluster_size, linear_num_calcs, '(15,30)'))

### Clusters with rod-calcs with vertical and no smoothing

In [11]:
rod_cluster_size = 142
rod_num_calcs = 20
create_cluster_rod(rod_cluster_size, rod_num_calcs).tofile('./calc_cluster_examples/cluster_rod{0}x{0}_nm{1}.raw'.format(rod_cluster_size, 
                                                                                                            rod_num_calcs))

### Clusters with rod-like and spherical calcs 

In [12]:
both_cluster_size = 142
both_num_calcs = 20
create_cluster_rod_sphere(both_cluster_size, both_num_calcs).tofile('cluster_red_sphere{0}x{0}_nm{1}.raw'.format(both_cluster_size,
                                                                                                             both_num_calcs))

# Placing calcifications inside breast phantom
## Placing 5 (71 vx), 10 (142 vx), and 20 (285 vx) mm sizes cluster
Specification for MC-GPU: 1 vx^3 = 70 micromm.
Breast phantom size is 1680 x 2159 x 601 voxels. Each phantom will have 12 clusters with 3 rows by 4 columns. The example here places spherical calcifications in random clusters.

Zipped examples of 5, 10, and 20 mm clusters with 40 calcs (randomly placed and spherical) in phantoms are in the folder `/phantom`.

**Note**: For placing 20 mm (285 vx) sized clusters side by side, horizontal space between each cluster should be 450 voxels instead of 350. Specifically for 20 mm sized clusters, I placed 9 clusters in each phantom instead of 12 for 3 rows x 3 cols where all of them were 20 mm and changed the horizontal space to be 450 voxels. See code below for example of placing only 20 mm clusters.

```python
#Number of rows of clusters.
for ii in range(3): 

    #Beginning x-coordinate for the first cluster on each row. 
    change_x=x_cent-600 

    #Number of clusters in each row. 
    for iii in range(3): 

        #Creating cluster (can change which type by changing the function).
        cluster = create_cluster(cluster_size, num_calcs, min_calc_size=3, max_calc_size=9)
        for i in range(cluster_size):
            for x in range(cluster_size):
                for y in range(cluster_size):
                    if cluster[i,x,y] == 1:
                        bb[start_i+i, change_x+x, start_y+y]=250

        #For placing 20 mm (285 vx) sized clusters side by side, 
        #horizontal space between each cluster should be 450 voxels instead of 350.
        change_x=change_x+450

    #Vertical space between each cluster is 400 voxels.
    start_y = start_y+400
```

In [13]:
#Center of breast phantom.
x_cent=2159//2
y_cent=1680//2
i_cent=601//2

phantom_name="cluster_phantom_{0}_{1}nm.raw"

#Unzip Graff breast phantom.
#Path to breast phantom.
path = '/phantom/pce_1764975963_crop.raw'

#List of parameters for the clusters placed in each phantom.
#Each tuple corresponds to one phantom. 
#This list will have 10 mm (142 vx) and 5 mm (71 vx) clusters with 40 calcs in each. 
parameters_list = [(142, 40), (71, 40)]

for i in range(len(parameters_list)): 
   
    #Variables for the cluster size and number of calcs.
    parameters = parameters_list[i]
    cluster_size = parameters[0]
    num_calcs = parameters[1]
    
    #The first slice we begin inserting the cluster. 
    #Start at the middle slice - half of the size of the cluster, so center slice of cluster will be at 
    #same as center slice for the breast phantom.
    start_i = i_cent-cluster_size//2 
    
    #Loading in breast phantom. 
    bb = np.fromfile(path, dtype='uint8')
    bb=bb.reshape(601, 2159, 1680)
    
    #Beginning y-coordinate for the first cluster. 
    start_y=y_cent - 700 
    
    #Number of rows of clusters.
    for ii in range(3): 
        
        #Beginning x-coordinate for the first cluster on each row. 
        change_x=x_cent-600 
        
        #Number of clusters in each row. 
        for iii in range(4): 
            
            #Creating cluster (can change which type by changing the function).
            cluster = create_cluster(cluster_size, num_calcs, min_calc_size=3, max_calc_size=9)
            for i in range(cluster_size):
                for x in range(cluster_size):
                    for y in range(cluster_size):
                        if cluster[i,x,y] == 1:
                            bb[start_i+i, change_x+x, start_y+y]=250
                            
            #Horizontal space between each cluster is 350 voxels.
            #For placing 20 mm (285 vx) sized clusters side by side, 
            #horizontal space between each cluster should be 450 voxels instead of 350.
            change_x=change_x+350
            
        #Vertical space between each cluster is 400 voxels.
        start_y = start_y+400
    bb.tofile('./phantom/'+phantom_name.format(cluster_size, num_calcs))
    print(phantom_name.format(cluster_size, num_calcs))

FileNotFoundError: [Errno 2] No such file or directory: '/phantom/pce_1764975963_crop.raw'

# Pre-processing after MC-GPU
After running MC-GPU on the breast phantoms, the preprocess file contains the functions for processing the .RAW MC-GPU output files and places them inside `/post_images` folder. MC-GPU outputs of the examples of breast phantoms created above are in the `/mcgpu_output` folder, and post-processing images for those examples are also located in the `/post_images` folder.

Processing code was modified from code written by Dr. Kenny Cha.

In [1]:
from preprocessing import * #File in this repo 

For processing all images in folder `/mcgpu_output` images at once.

In [2]:
path = pathlib.Path('./mcgpu_output/')
for currentFile in path.iterdir():
    if currentFile.suffix == '.raw':
        print(currentFile)
        
        #Uses default values of the function preprocess.
        test = preprocess(currentFile, other=400000, lower=25, upper=100)
        
        #Inserts brackets after file extensions. Ex. prj_30mm_2_cluster_malignant_only_142_10nm_s{}.raw.gz.raw
        #Bracket will be replaced with string value in write_out function that includes more information. 
        img_name = currentFile.name[:-11] + '{}' + currentFile.name[-11:]
        
        #Choosing to use the values greater than 0.7 and keeping the 25th to 100th percentile.
        write_out(test, img_name, path='./post_images/', add='_full_0.7_25')
        print(img_name)

mcgpu_output/prj_30mm_2_cluster_phantom_285_40nm_m.raw.gz.raw
prj_30mm_2_cluster_phantom_285_40nm_m{}.raw.gz.raw
mcgpu_output/prj_30mm_2_cluster_phantom_71_40nm_m.raw.gz.raw
prj_30mm_2_cluster_phantom_71_40nm_m{}.raw.gz.raw
mcgpu_output/prj_30mm_2_cluster_phantom_142_40nm.raw.gz.raw
prj_30mm_2_cluster_phantom_142_40nm{}.raw.gz.raw
